# 🧠 QNN Codelab: Quantum Neural Networks

## Building and Training Variational Quantum Classifiers

| Property | Value |
|----------|-------|
| **Algorithm** | Quantum Neural Network (Variational Classifier) |
| **Difficulty** | 🔴 Advanced |
| **Time** | 90-120 minutes |
| **Prerequisites** | Module-12-Quantum-ML.md, VQE concepts |
| **Qiskit Version** | 2.x |

---

## 🎯 Learning Objectives

By completing this codelab, you will:

1. ✅ Implement feature maps for data encoding
2. ✅ Build variational ansätze with trainable parameters
3. ✅ Compute gradients using the parameter shift rule
4. ✅ Train a QNN classifier on toy datasets
5. ✅ Compare QNN performance to classical methods

---

## Section 1: Environment Setup

In [ ]:
# Core imports
import numpy as np
import matplotlib.pyplot as plt
from numpy import pi

# Qiskit imports
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from qiskit.primitives import StatevectorSampler
from qiskit_ibm_runtime.fake_provider import FakeAlmadenV2
from qiskit.quantum_info import Statevector, Operator
from qiskit.visualization import plot_histogram, plot_bloch_multivector

# Install scikit-learn
%pip install scikit-learn

# ML tools
from sklearn.datasets import make_moons, make_circles
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
# Initialize fake backend and sampler
fake_backend = FakeAlmadenV2()
sampler = StatevectorSampler()

# Version check
import qiskit
print(f"Qiskit version: {qiskit.__version__}")
assert int(qiskit.__version__.split('.')[0]) >= 1, "Requires Qiskit 1.x or 2.x"

# Configure plotting
plt.rcParams['figure.figsize'] = (10, 6)

plt.rcParams['font.size'] = 12
print("✅ All imports successful!")

np.random.seed(42)

---

## Section 2: Theory Recap

### QNN Architecture

$$f_\theta(x) = \langle 0 | U^\dagger(x) V^\dagger(\theta) Z V(\theta) U(x) | 0 \rangle$$

Where:
- $U(x)$: Feature map (encodes input data)
- $V(\theta)$: Variational ansatz (trainable)
- $Z$: Measurement observable

### Parameter Shift Rule

For gradient computation:
$$\frac{\partial f}{\partial \theta} = \frac{f(\theta + \pi/2) - f(\theta - \pi/2)}{2}$$

### Training

Minimize loss: $\mathcal{L} = \sum_i (y_i - f_\theta(x_i))^2$

---

## Section 3: Create Toy Datasets

In [ ]:
def create_datasets():
    """
    Create toy classification datasets.
    
    Returns:
        dict: Datasets with train/test splits
    """
    datasets = {}
    
    # Moons dataset (non-linear)
    X_moons, y_moons = make_moons(n_samples=200, noise=0.1, random_state=42)
    
    # Scale to [0, π] for angle encoding
    scaler = MinMaxScaler(feature_range=(0, np.pi))
    X_moons_scaled = scaler.fit_transform(X_moons)
    
    # Convert labels to {-1, +1}
    y_moons_binary = 2 * y_moons - 1
    
    X_train, X_test, y_train, y_test = train_test_split(
        X_moons_scaled, y_moons_binary, test_size=0.3, random_state=42
    )
    
    datasets['moons'] = {
        'X_train': X_train,
        'X_test': X_test,
        'y_train': y_train,
        'y_test': y_test,
        'X_full': X_moons_scaled,
        'y_full': y_moons_binary
    }
    
    # Linearly separable dataset (for sanity check)
    np.random.seed(42)
    n_samples = 100
    X_linear = np.random.randn(n_samples, 2)
    y_linear = np.sign(X_linear[:, 0] + X_linear[:, 1])  # Line x + y = 0
    y_linear[y_linear == 0] = 1  # Handle edge case
    
    X_linear_scaled = scaler.fit_transform(X_linear)
    
    X_train_lin, X_test_lin, y_train_lin, y_test_lin = train_test_split(
        X_linear_scaled, y_linear, test_size=0.3, random_state=42
    )
    
    datasets['linear'] = {
        'X_train': X_train_lin,
        'X_test': X_test_lin,
        'y_train': y_train_lin,
        'y_test': y_test_lin
    }
    
    return datasets

# Create datasets
datasets = create_datasets()

# Visualize
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

for ax, (name, data) in zip(axes, datasets.items()):
    if 'X_full' in data:
        X, y = data['X_full'], data['y_full']
    else:
        X = np.vstack([data['X_train'], data['X_test']])
        y = np.hstack([data['y_train'], data['y_test']])
    
    ax.scatter(X[y == 1, 0], X[y == 1, 1], c='blue', marker='o', label='Class +1')
    ax.scatter(X[y == -1, 0], X[y == -1, 1], c='red', marker='x', label='Class -1')
    ax.set_xlabel('Feature 1')
    ax.set_ylabel('Feature 2')
    ax.set_title(f'{name.capitalize()} Dataset')
    ax.legend()

plt.tight_layout()
plt.show()

print(f"\n📊 Dataset Statistics:")
for name, data in datasets.items():
    print(f"  {name}: {len(data['X_train'])} train, {len(data['X_test'])} test samples")

---

## Section 4: Feature Map Implementation

In [ ]:
def angle_encoding(qc: QuantumCircuit, x: np.ndarray) -> None:
    """
    Simple angle encoding: each feature becomes a rotation angle.
    
    Parameters:
        qc: QuantumCircuit to modify
        x: Input features (array of length n_qubits)
    """
    n_qubits = len(x)
    for i in range(n_qubits):
        qc.ry(x[i], i)

def zz_feature_map(qc: QuantumCircuit, x: np.ndarray, reps: int = 2) -> None:
    """
    ZZ Feature Map with entanglement.
    
    Encodes data with single-qubit rotations and ZZ entanglement.
    
    Parameters:
        qc: QuantumCircuit to modify
        x: Input features
        reps: Number of repetitions
    """
    n_qubits = len(x)
    
    for _ in range(reps):
        # Hadamard layer
        for i in range(n_qubits):
            qc.h(i)
        
        # Single-qubit rotation layer
        for i in range(n_qubits):
            qc.rz(2 * x[i], i)
        
        # ZZ entanglement layer
        for i in range(n_qubits - 1):
            qc.cx(i, i + 1)
            qc.rz(2 * x[i] * x[i + 1], i + 1)
            qc.cx(i, i + 1)

# Demonstrate feature maps
x_sample = np.array([np.pi/4, np.pi/3])

# Angle encoding
qc_angle = QuantumCircuit(2)
angle_encoding(qc_angle, x_sample)

print("Angle Encoding:")
print(qc_angle.draw(output='text'))

# ZZ feature map
qc_zz = QuantumCircuit(2)
zz_feature_map(qc_zz, x_sample, reps=1)

print("\nZZ Feature Map (1 rep):")
print(qc_zz.draw(output='text'))

# Compare resulting states
print("\n📊 Resulting States:")
print(f"  Angle encoding: {Statevector(qc_angle).data.round(3)}")
print(f"  ZZ feature map: {Statevector(qc_zz).data.round(3)}")

---

## Section 5: Variational Ansatz

In [ ]:
def variational_layer(qc: QuantumCircuit, params: np.ndarray, n_qubits: int) -> None:
    """
    Single variational layer with rotations and entanglement.
    
    Parameters:
        qc: QuantumCircuit to modify
        params: Array of 3*n_qubits parameters [Rx, Ry, Rz for each qubit]
        n_qubits: Number of qubits
    """
    # Rotation layer (Rx, Ry, Rz per qubit)
    for i in range(n_qubits):
        qc.rx(params[3*i], i)
        qc.ry(params[3*i + 1], i)
        qc.rz(params[3*i + 2], i)
    
    # Entanglement layer (CNOT chain)
    for i in range(n_qubits - 1):
        qc.cx(i, i + 1)
    # Wrap around for cyclic entanglement
    if n_qubits > 2:
        qc.cx(n_qubits - 1, 0)

def variational_ansatz(qc: QuantumCircuit, params: np.ndarray, n_qubits: int, n_layers: int) -> None:
    """
    Multi-layer variational ansatz.
    
    Parameters:
        qc: QuantumCircuit to modify
        params: All parameters (3 * n_qubits * n_layers total)
        n_qubits: Number of qubits
        n_layers: Number of variational layers
    """
    params_per_layer = 3 * n_qubits
    
    for layer in range(n_layers):
        layer_params = params[layer * params_per_layer : (layer + 1) * params_per_layer]
        variational_layer(qc, layer_params, n_qubits)

# Demonstrate ansatz
n_qubits = 2
n_layers = 2
n_params = 3 * n_qubits * n_layers

test_params = np.random.uniform(0, 2*np.pi, n_params)

qc_ansatz = QuantumCircuit(n_qubits)
variational_ansatz(qc_ansatz, test_params, n_qubits, n_layers)

print(f"Variational Ansatz ({n_layers} layers, {n_params} parameters):")
print(qc_ansatz.draw(output='text'))

print(f"\n📊 Ansatz Statistics:")
print(f"  Qubits: {n_qubits}")
print(f"  Layers: {n_layers}")
print(f"  Parameters: {n_params}")
print(f"  Circuit depth: {qc_ansatz.depth()}")

---

## Section 5.5: State Evolution Analysis

### 5.5.1 The "Feature Map + Variational Layer" Rule

**The Key Identity**: For a quantum neural network classifier:

$$f_\theta(x) = \langle 0 | U^\dagger(x) V^\dagger(\theta) M V(\theta) U(x) | 0 \rangle$$

Where:
- $U(x)$: **Feature map** - encodes classical data $x$ into quantum state
- $V(\theta)$: **Variational ansatz** - trainable quantum circuit with parameters $\theta$
- $M$: **Measurement operator** - typically Pauli Z on first qubit

**The Rule**: "Encode data into quantum amplitudes/phases, transform with trainable rotations, extract prediction via measurement."

**From the Lecture**:
> "Think of this like VQE type of circuit, where I will perform unitary rotations on a bunch of qubits. And the amount of rotation is given by some parameters θ. These θ could serve the purpose of weight matrix in classical neural networks."

**Why This Works**:

1. **Feature map creates data-dependent state**: $|x\rangle = U(x)|0\rangle$
2. **Variational layer transforms**: $|\psi(\theta)\rangle = V(\theta)|x\rangle$  
3. **Measurement extracts prediction**: $f = \langle\psi|M|\psi\rangle \in [-1, 1]$
4. **Non-linearity from measurement**: $P(0) = |\langle 0|\psi\rangle|^2$ is non-linear!

### 5.5.2 State Evolution Through a QNN

| Stage | State | Physical Meaning |
|-------|-------|------------------|
| **Initial** | $\|0\rangle^{\otimes n}$ | Blank quantum register |
| **After Feature Map** | $\|x\rangle = U(x)\|0\rangle$ | Data encoded as quantum state |
| **After Ansatz** | $\|\psi(\theta)\rangle = V(\theta)\|x\rangle$ | Trainable transformation applied |
| **Measurement** | $\langle Z\rangle = \langle\psi\|Z\|\psi\rangle$ | Prediction extracted as expectation value |

**Key Insight from Lecture**:
> "All these unitary transformations are linear transformations. But I do need some non-linearity like that neuron. That nonlinearity comes eventually by final measurement. Measurement collapses the state—it is not a linear transformation."

**Why Measurement Provides Non-Linearity**:
- Unitary operations: $|\psi\rangle \to U|\psi\rangle$ (linear in amplitudes)
- Measurement probability: $P(0) = |\alpha|^2$ (non-linear in amplitudes)
- This is analogous to activation functions in classical NNs!

In [ ]:
def trace_qnn_evolution(
    x: np.ndarray, 
    params: np.ndarray, 
    n_qubits: int = 2, 
    n_layers: int = 1
) -> None:
    """
    Trace state evolution through a QNN step by step.
    
    Demonstrates the "Feature Map + Variational Layer" rule.
    
    Parameters:
        x: Input features
        params: Variational parameters
        n_qubits: Number of qubits
        n_layers: Number of variational layers
    """
    print("=" * 70)
    print("QNN STATE EVOLUTION TRACE")
    print("=" * 70)
    
    print(f"\n📥 INPUT:")
    print(f"   Classical data x = {x}")
    print(f"   Variational parameters θ = {params[:6]}..." if len(params) > 6 else f"   θ = {params}")
    
    # Stage 1: Initial state
    qc = QuantumCircuit(n_qubits)
    state_0 = Statevector(qc)
    print(f"\n🔵 STAGE 1: Initial State |0⟩^⊗{n_qubits}")
    print(f"   |ψ₀⟩ = {state_0.data}")
    
    # Stage 2: After feature map
    angle_encoding(qc, x)
    state_x = Statevector(qc)
    print(f"\n🟢 STAGE 2: After Feature Map U(x)")
    print(f"   Applied: R_Y(x_i) on each qubit")
    print(f"   |x⟩ = U(x)|0⟩")
    print(f"   Amplitudes: {np.round(state_x.data, 4)}")
    
    # Analyze feature encoding
    print(f"\n   📊 Feature Encoding Analysis:")
    for i, xi in enumerate(x):
        print(f"      Qubit {i}: R_Y({xi:.4f}) rotates |0⟩ toward |1⟩")
        print(f"               Creates cos({xi/2:.3f})|0⟩ + sin({xi/2:.3f})|1⟩")
    
    # Stage 3: After variational ansatz
    qc_full = QuantumCircuit(n_qubits)
    angle_encoding(qc_full, x)
    variational_ansatz(qc_full, params, n_qubits, n_layers)
    state_theta = Statevector(qc_full)
    
    print(f"\n🟡 STAGE 3: After Variational Ansatz V(θ)")
    print(f"   |ψ(θ)⟩ = V(θ)|x⟩")
    print(f"   Amplitudes: {np.round(state_theta.data, 4)}")
    
    # Compute expectation value
    Z = np.array([[1, 0], [0, -1]])
    identity = np.eye(2)
    
    # Z on first qubit, identity on rest
    Z_full = Z
    for _ in range(n_qubits - 1):
        Z_full = np.kron(identity, Z_full)
    
    exp_z = np.real(state_theta.data.conj() @ Z_full @ state_theta.data)
    
    print(f"\n🔴 STAGE 4: Measurement ⟨Z₀⟩")
    print(f"   Observable: Z on qubit 0")
    print(f"   ⟨Z⟩ = ⟨ψ(θ)|Z|ψ(θ)⟩ = {exp_z:.4f}")
    print(f"   Prediction: {'Class +1' if exp_z > 0 else 'Class -1'}")
    
    # Show Bloch sphere for 2-qubit case
    if n_qubits == 2:
        print(f"\n📈 PROBABILITY DISTRIBUTION:")
        probs = state_theta.probabilities()
        for i, p in enumerate(probs):
            basis = format(i, f'0{n_qubits}b')
            bar = '█' * int(p * 40)
            print(f"   |{basis}⟩: {p:.4f} {bar}")
    
    return exp_z

# Demonstrate with sample data
print("Tracing QNN evolution for a single input point...\n")

x_demo = np.array([np.pi/4, np.pi/3])  # Sample input
params_demo = np.random.uniform(0, np.pi, 3 * 2 * 1)  # 2 qubits, 1 layer

prediction = trace_qnn_evolution(x_demo, params_demo, n_qubits=2, n_layers=1)

### 5.5.3 Understanding Feature Maps: How Data Becomes Quantum

Different encoding strategies create different quantum "feature spaces":

| Encoding | Transformation | Use Case |
|----------|----------------|----------|
| **Angle** | $R_Y(x_i)\|0\rangle$ | Simple, efficient, 1 qubit per feature |
| **Amplitude** | $\sum_i x_i\|i\rangle$ | Efficient for high-dim data, needs normalization |
| **ZZ** | $H \cdot R_Z(x) \cdot ZZ(x_i x_j)$ | Captures feature interactions |
| **Basis** | $\|x\rangle$ for binary $x$ | Discrete/binary data |

In [ ]:
def compare_feature_maps(x: np.ndarray) -> None:
    """
    Compare how different feature maps encode the same data point.
    
    From the lecture:
    "At the input I give some input state whose features I want to learn. 
    This could be a classical vector loaded inside the quantum computer as a quantum state."
    """
    print("=" * 70)
    print("FEATURE MAP COMPARISON")
    print("=" * 70)
    
    print(f"\n📥 Input data: x = {x}")
    print(f"   (Scaled to [0, π] for encoding)")
    
    n_qubits = len(x)
    
    # 1. Angle encoding
    qc_angle = QuantumCircuit(n_qubits)
    angle_encoding(qc_angle, x)
    state_angle = Statevector(qc_angle)
    
    print(f"\n🔵 ANGLE ENCODING: R_Y(x_i)|0⟩")
    print(f"   Circuit depth: {qc_angle.depth()}")
    print(f"   State: {np.round(state_angle.data, 4)}")
    
    # 2. ZZ feature map
    qc_zz = QuantumCircuit(n_qubits)
    zz_feature_map(qc_zz, x, reps=1)
    state_zz = Statevector(qc_zz)
    
    print(f"\n🟢 ZZ FEATURE MAP: H → R_Z(x) → ZZ(x_i·x_j)")
    print(f"   Circuit depth: {qc_zz.depth()}")
    print(f"   State: {np.round(state_zz.data, 4)}")
    
    # 3. Compute inner products (quantum kernels)
    # K(x, x') = |⟨φ(x)|φ(x')⟩|²
    
    # Compare with shifted data point
    x_shifted = x + 0.3
    
    # Angle kernel
    qc_angle_shifted = QuantumCircuit(n_qubits)
    angle_encoding(qc_angle_shifted, x_shifted)
    state_angle_shifted = Statevector(qc_angle_shifted)
    kernel_angle = np.abs(np.vdot(state_angle.data, state_angle_shifted.data))**2
    
    # ZZ kernel
    qc_zz_shifted = QuantumCircuit(n_qubits)
    zz_feature_map(qc_zz_shifted, x_shifted, reps=1)
    state_zz_shifted = Statevector(qc_zz_shifted)
    kernel_zz = np.abs(np.vdot(state_zz.data, state_zz_shifted.data))**2
    
    print(f"\n📊 QUANTUM KERNEL: K(x, x') = |⟨φ(x)|φ(x')⟩|²")
    print(f"   (Measuring similarity between x and x + 0.3)")
    print(f"   Angle encoding kernel: K = {kernel_angle:.4f}")
    print(f"   ZZ feature map kernel: K = {kernel_zz:.4f}")
    
    # Visualize
    fig, axes = plt.subplots(1, 3, figsize=(15, 4))
    
    # Probabilities for angle encoding
    ax1 = axes[0]
    probs_angle = state_angle.probabilities()
    bars = ax1.bar(range(len(probs_angle)), probs_angle, color='steelblue')
    ax1.set_xlabel('Basis state')
    ax1.set_ylabel('Probability')
    ax1.set_title('Angle Encoding')
    ax1.set_xticks(range(len(probs_angle)))
    ax1.set_xticklabels([format(i, f'0{n_qubits}b') for i in range(len(probs_angle))])
    
    # Probabilities for ZZ encoding
    ax2 = axes[1]
    probs_zz = state_zz.probabilities()
    bars = ax2.bar(range(len(probs_zz)), probs_zz, color='coral')
    ax2.set_xlabel('Basis state')
    ax2.set_ylabel('Probability')
    ax2.set_title('ZZ Feature Map')
    ax2.set_xticks(range(len(probs_zz)))
    ax2.set_xticklabels([format(i, f'0{n_qubits}b') for i in range(len(probs_zz))])
    
    # Kernel comparison
    ax3 = axes[2]
    x_range = np.linspace(0, np.pi, 20)
    kernels_angle = []
    kernels_zz = []
    
    for x_new in x_range:
        x_test = np.array([x_new, x[1]])  # Vary first feature
        
        qc_test = QuantumCircuit(n_qubits)
        angle_encoding(qc_test, x_test)
        k = np.abs(np.vdot(state_angle.data, Statevector(qc_test).data))**2
        kernels_angle.append(k)
        
        qc_test_zz = QuantumCircuit(n_qubits)
        zz_feature_map(qc_test_zz, x_test, reps=1)
        k_zz = np.abs(np.vdot(state_zz.data, Statevector(qc_test_zz).data))**2
        kernels_zz.append(k_zz)
    
    ax3.plot(x_range, kernels_angle, 'b-', linewidth=2, label='Angle')
    ax3.plot(x_range, kernels_zz, 'r--', linewidth=2, label='ZZ')
    ax3.axvline(x=x[0], color='gray', linestyle=':', alpha=0.5, label=f'x₁={x[0]:.2f}')
    ax3.set_xlabel('x₁ value')
    ax3.set_ylabel('Kernel K(x, x\')')
    ax3.set_title('Kernel Similarity Decay')
    ax3.legend()
    ax3.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print("\n🔑 KEY INSIGHT:")
    print("   - ZZ feature map creates more complex kernel due to entanglement")
    print("   - Feature maps define the 'quantum feature space' for classification")
    print("   - Different maps work better for different data distributions")

compare_feature_maps(np.array([np.pi/4, np.pi/3]))

### 5.5.4 Interactive: Explore the Loss Landscape

The **loss function** drives training:
$$\mathcal{L}(\theta) = \sum_i \left( y_i - f_\theta(x_i) \right)^2$$

From the lecture:
> "Given the parameters θ and the input state, if this expectation value is exactly same as the label L, then the loss will be 0. But in reality we will measure some loss because this expectation value will be deviated away from this ideal value. Our job is to minimize this loss."

In [ ]:
def explore_qnn_loss_landscape():
    """
    Visualize the loss landscape of a QNN for a simple classification task.
    
    Demonstrates barren plateaus and local minima.
    """
    print("=" * 70)
    print("QNN LOSS LANDSCAPE EXPLORATION")
    print("=" * 70)
    
    # Simple 2-point classification
    X_simple = np.array([[0.5, 0.5], [2.5, 2.5]])  # Two points
    y_simple = np.array([1, -1])  # Class +1 and -1
    
    print(f"\n📥 Simple classification task:")
    print(f"   Point 1: x={X_simple[0]}, y=+1")
    print(f"   Point 2: x={X_simple[1]}, y=-1")
    
    # For 2 qubits, 1 layer: 6 parameters
    # Fix 4 parameters, vary 2 to visualize
    n_qubits = 2
    n_layers = 1
    n_params = 3 * n_qubits * n_layers
    
    base_params = np.array([np.pi/4, np.pi/4, np.pi/4, np.pi/4, np.pi/4, np.pi/4])
    
    # Create meshgrid for first two parameters
    theta_range = np.linspace(0, 2*np.pi, 25)
    theta1, theta2 = np.meshgrid(theta_range, theta_range)
    
    print(f"\n⏳ Computing loss landscape (varying θ₀ and θ₁)...")
    
    loss_landscape = np.zeros_like(theta1)
    
    for i in range(theta1.shape[0]):
        for j in range(theta1.shape[1]):
            params = base_params.copy()
            params[0] = theta1[i, j]
            params[1] = theta2[i, j]
            
            # Compute loss for both points
            total_loss = 0
            for x, y in zip(X_simple, y_simple):
                # Use statevector for fast evaluation
                qc = QuantumCircuit(n_qubits)
                angle_encoding(qc, x)
                variational_ansatz(qc, params, n_qubits, n_layers)
                state = Statevector(qc)
                
                # Compute ⟨Z⟩ on first qubit
                probs = state.probabilities()
                # P(0 on first qubit) = sum of probs where first bit is 0
                p0 = probs[0] + probs[2] if n_qubits == 2 else probs[0]
                p1 = probs[1] + probs[3] if n_qubits == 2 else probs[1]
                exp_z = p0 - p1
                
                total_loss += (y - exp_z)**2
            
            loss_landscape[i, j] = total_loss / 2
    
    # Visualize
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # 3D surface
    from mpl_toolkits.mplot3d import Axes3D
    ax1 = fig.add_subplot(121, projection='3d')
    ax1.plot_surface(theta1, theta2, loss_landscape, cmap='viridis', alpha=0.8)
    ax1.set_xlabel('θ₀')
    ax1.set_ylabel('θ₁')
    ax1.set_zlabel('Loss')
    ax1.set_title('QNN Loss Landscape (3D)')
    
    # Contour plot
    ax2 = axes[1]
    contour = ax2.contourf(theta1, theta2, loss_landscape, levels=20, cmap='viridis')
    plt.colorbar(contour, ax=ax2, label='Loss')
    ax2.set_xlabel('θ₀')
    ax2.set_ylabel('θ₁')
    ax2.set_title('QNN Loss Landscape (Contour)')
    
    # Mark minimum
    min_idx = np.unravel_index(np.argmin(loss_landscape), loss_landscape.shape)
    ax2.plot(theta1[min_idx], theta2[min_idx], 'r*', markersize=15, label=f'Min Loss: {loss_landscape.min():.3f}')
    ax2.legend()
    
    plt.tight_layout()
    plt.show()
    
    print(f"\n📊 Landscape Statistics:")
    print(f"   Minimum loss: {loss_landscape.min():.4f} at θ₀={theta1[min_idx]:.2f}, θ₁={theta2[min_idx]:.2f}")
    print(f"   Maximum loss: {loss_landscape.max():.4f}")
    print(f"   Mean loss: {loss_landscape.mean():.4f}")
    
    print("\n🔑 KEY INSIGHT (Barren Plateaus):")
    print("   For deeper circuits and more qubits, the landscape becomes")
    print("   increasingly flat → gradients vanish → training becomes hard!")
    print("   This is why shallow QNNs often work better than deep ones.")

explore_qnn_loss_landscape()

### 5.5.5 The Barren Plateau Problem: Why Deep QNNs Are Hard to Train

From the lecture:
> "Whether there's an advantage over classical neural networks—that prediction is still open. Most likely for classical type of tasks, the classical neural networks are already very well advanced."

**The Problem**: For random parameterized circuits, gradient variance scales as:
$$\text{Var}\left[\frac{\partial \mathcal{L}}{\partial \theta}\right] \sim \frac{1}{2^n}$$

This means gradients become **exponentially small** with more qubits!

In [ ]:
def demonstrate_barren_plateaus():
    """
    Demonstrate how gradient variance decreases with circuit complexity.
    
    The key insight from QML research: random circuits lead to barren plateaus
    where gradients vanish exponentially with qubit count.
    """
    print("=" * 70)
    print("BARREN PLATEAU DEMONSTRATION")
    print("=" * 70)
    
    def estimate_gradient_variance_exact(n_qubits, n_layers, n_samples=20):
        """Estimate gradient variance using statevector simulation."""
        n_params = 3 * n_qubits * n_layers
        x_sample = np.random.uniform(0, np.pi, n_qubits)
        
        gradients = []
        for _ in range(n_samples):
            params = np.random.uniform(0, 2*np.pi, n_params)
            
            # Compute gradient of first parameter using parameter shift
            params_plus = params.copy()
            params_plus[0] += np.pi / 2
            params_minus = params.copy()
            params_minus[0] -= np.pi / 2
            
            # Forward pass +
            qc_plus = QuantumCircuit(n_qubits)
            angle_encoding(qc_plus, x_sample)
            variational_ansatz(qc_plus, params_plus, n_qubits, n_layers)
            state_plus = Statevector(qc_plus)
            probs_plus = state_plus.probabilities()
            exp_z_plus = sum((-1)**((i >> 0) & 1) * p for i, p in enumerate(probs_plus))
            
            # Forward pass -
            qc_minus = QuantumCircuit(n_qubits)
            angle_encoding(qc_minus, x_sample)
            variational_ansatz(qc_minus, params_minus, n_qubits, n_layers)
            state_minus = Statevector(qc_minus)
            probs_minus = state_minus.probabilities()
            exp_z_minus = sum((-1)**((i >> 0) & 1) * p for i, p in enumerate(probs_minus))
            
            grad = (exp_z_plus - exp_z_minus) / 2
            gradients.append(grad)
        
        return np.var(gradients), np.mean(np.abs(gradients))
    
    # Test different configurations
    configs = [
        (2, 1, "2 qubits, 1 layer"),
        (2, 2, "2 qubits, 2 layers"),
        (2, 4, "2 qubits, 4 layers"),
        (3, 2, "3 qubits, 2 layers"),
        (4, 2, "4 qubits, 2 layers"),
    ]
    
    print(f"\n{'Configuration':<25} {'Var(∂f/∂θ)':<15} {'Mean |∇|':<15} {'Trainable?':<12}")
    print("-" * 70)
    
    results = []
    for n_q, n_l, desc in configs:
        var, mean_abs = estimate_gradient_variance_exact(n_q, n_l, n_samples=15)
        trainable = "✓ Yes" if var > 0.001 else "⚠ Difficult" if var > 0.0001 else "✗ Very Hard"
        print(f"{desc:<25} {var:<15.6f} {mean_abs:<15.6f} {trainable:<12}")
        results.append((desc, var, mean_abs))
    
    # Visualization
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Gradient variance vs config
    ax1 = axes[0]
    labels = [r[0] for r in results]
    variances = [r[1] for r in results]
    ax1.bar(range(len(labels)), variances, color='steelblue')
    ax1.set_xticks(range(len(labels)))
    ax1.set_xticklabels(labels, rotation=45, ha='right')
    ax1.set_ylabel('Gradient Variance')
    ax1.set_title('Gradient Variance vs Circuit Complexity')
    ax1.set_yscale('log')
    ax1.axhline(y=0.001, color='orange', linestyle='--', label='Trainability threshold')
    ax1.legend()
    
    # Theoretical scaling
    ax2 = axes[1]
    n_qubits_range = np.arange(2, 10)
    theoretical_var = 1 / (2 ** n_qubits_range)  # Theoretical: Var ~ 1/2^n
    
    ax2.semilogy(n_qubits_range, theoretical_var, 'r-o', linewidth=2, markersize=8, label='Theoretical: 1/2ⁿ')
    ax2.set_xlabel('Number of Qubits')
    ax2.set_ylabel('Gradient Variance')
    ax2.set_title('Barren Plateau: Exponential Vanishing')
    ax2.grid(True, alpha=0.3)
    ax2.legend()
    
    # Annotate
    for n, v in zip(n_qubits_range, theoretical_var):
        if n in [2, 5, 8]:
            ax2.annotate(f'{v:.1e}', (n, v), textcoords="offset points", xytext=(5, 5))
    
    plt.tight_layout()
    plt.show()
    
    print("\n🔑 KEY INSIGHTS:")
    print("   1. Deeper circuits → smaller gradient variance → harder training")
    print("   2. More qubits → exponentially smaller gradients (barren plateaus)")
    print("   3. Mitigation strategies:")
    print("      - Use shallow circuits (few layers)")
    print("      - Problem-specific ansätze (not random)")
    print("      - Layer-wise training")
    print("      - Parameter initialization near identity")

demonstrate_barren_plateaus()

### 5.5.6 Summary: How QNNs Work (and When They Might Help)

| Component | Formula | Role in QNN |
|-----------|---------|-------------|
| **Feature Map** | $U(x)\|0\rangle$ | Encodes classical data into quantum state |
| **Variational Layer** | $V(\theta)$ | Trainable transformation (like weights in classical NN) |
| **Measurement** | $\langle Z \rangle$ | Extracts prediction; provides non-linearity |
| **Loss Function** | $\mathcal{L} = \sum_i (y_i - f_\theta(x_i))^2$ | Drives optimization via gradient descent |
| **Gradient** | Parameter shift: $\frac{f(\theta+\pi/2) - f(\theta-\pi/2)}{2}$ | Exact gradient without finite differences |

**The "Feature Map + Variational Layer" Rule**:
> "Encode data with U(x), apply trainable rotations V(θ), measure to get prediction. Optimize θ to minimize loss. The key is that quantum feature spaces may separate data that classical methods cannot."

**Key Insights from the Lecture**:

1. **Why measurement is crucial**: "Nonlinearity comes eventually by final measurement. Measurement collapses the state—it is not a linear transformation."

2. **Quantum data advantage**: "When your input is quantum in nature—produced by some quantum computer or molecule or atom—such quantum circuits which are able to learn could provide a very useful tool."

3. **Hybrid approach**: "You can now start to combine quantum and classical systems together and see if the combined neural network that you get is good at doing something which just the classical neural network was not able to do."

**When QNNs Might Help**:
- ✅ **Quantum data**: Input comes from quantum sensors/simulations
- ✅ **Kernel advantage**: Quantum kernels separate data classical kernels cannot
- ✅ **Small data regimes**: Where overfitting to large models is a concern
- ✅ **As subroutines**: Within larger quantum algorithms

**Challenges**:
- ❌ **Barren plateaus**: Gradients vanish for deep/wide circuits
- ❌ **Classical baselines**: SVMs and NNs are already very powerful
- ❌ **Noise**: NISQ devices add errors to gradient estimates
- ❌ **No proven advantage**: For classical data, quantum speedup not established

---

## Section 6: Complete QNN Circuit

In [ ]:
def create_qnn_circuit(
    x: np.ndarray,
    params: np.ndarray,
    n_qubits: int,
    n_layers: int,
    feature_map: str = 'angle'
) -> QuantumCircuit:
    """
    Create complete QNN circuit.
    
    Parameters:
        x: Input features
        params: Variational parameters
        n_qubits: Number of qubits
        n_layers: Number of variational layers
        feature_map: 'angle' or 'zz'
    
    Returns:
        QuantumCircuit with QNN
    """
    qc = QuantumCircuit(n_qubits)
    
    # Feature map
    if feature_map == 'angle':
        angle_encoding(qc, x)
    elif feature_map == 'zz':
        zz_feature_map(qc, x, reps=1)
    else:
        raise ValueError(f"Unknown feature map: {feature_map}")
    
    qc.barrier()
    
    # Variational ansatz
    variational_ansatz(qc, params, n_qubits, n_layers)
    
    return qc

def qnn_expectation(
    x: np.ndarray,
    params: np.ndarray,
    n_qubits: int,
    n_layers: int,
    shots: int = 1024,
    feature_map: str = 'angle'
) -> float:
    """
    Compute QNN expectation value ⟨Z⟩ on first qubit.
    
    Parameters:
        x: Input features
        params: Variational parameters
        n_qubits: Number of qubits
        n_layers: Number of variational layers
        shots: Number of measurement shots
        feature_map: 'angle' or 'zz'
    
    Returns:
        float: ⟨Z⟩ expectation value in [-1, 1]
    """
    qc = create_qnn_circuit(x, params, n_qubits, n_layers, feature_map)
    qc.measure_all()
    
    # Transpile and run with SamplerV2
    transpiled_qc = transpile(qc, backend=fake_backend, optimization_level=3)
    job = sampler.run([transpiled_qc], shots=shots)
    result = job.result()
    counts = result[0].data.meas.get_counts()
    
    # Compute ⟨Z⟩ on first qubit (rightmost in Qiskit)
    exp_z = 0
    for bitstring, count in counts.items():
        # First qubit is rightmost
        bit = int(bitstring[-1])
        z_val = 1 - 2 * bit  # |0⟩ → +1, |1⟩ → -1
        exp_z += z_val * count
    
    return exp_z / shots

# Test QNN
x_test = np.array([np.pi/4, np.pi/3])
params_test = np.random.uniform(0, 2*np.pi, 3 * 2 * 2)  # 2 qubits, 2 layers

qc_full = create_qnn_circuit(x_test, params_test, n_qubits=2, n_layers=2, feature_map='angle')
print("Complete QNN Circuit:")
print(qc_full.draw(output='text'))

exp_val = qnn_expectation(x_test, params_test, n_qubits=2, n_layers=2, shots=1024)

print(f"\n⟨Z⟩ = {exp_val:.4f}")
print(f"Prediction: {'Class +1' if exp_val > 0 else 'Class -1'}")

---

## Section 7: Gradient Computation (Parameter Shift Rule)

In [ ]:
def compute_gradient(
    x: np.ndarray,
    params: np.ndarray,
    n_qubits: int,
    n_layers: int,
    shots: int = 1024,
    feature_map: str = 'angle'
) -> np.ndarray:
    """
    Compute gradient using parameter shift rule.
    
    ∂f/∂θ = (f(θ + π/2) - f(θ - π/2)) / 2
    
    Parameters:
        x: Input features
        params: Current parameters
        n_qubits, n_layers, shots, feature_map: QNN config
    
    Returns:
        np.ndarray: Gradient vector
    """
    n_params = len(params)
    gradient = np.zeros(n_params)
    
    for i in range(n_params):
        # f(θ + π/2)
        params_plus = params.copy()
        params_plus[i] += np.pi / 2
        f_plus = qnn_expectation(x, params_plus, n_qubits, n_layers, shots, feature_map)
        
        # f(θ - π/2)
        params_minus = params.copy()
        params_minus[i] -= np.pi / 2
        f_minus = qnn_expectation(x, params_minus, n_qubits, n_layers, shots, feature_map)
        
        gradient[i] = (f_plus - f_minus) / 2
    
    return gradient

# Demonstrate gradient computation
print("Computing gradient via parameter shift rule...")
grad = compute_gradient(x_test, params_test, n_qubits=2, n_layers=2, shots=2048)

print(f"\nGradient vector ({len(grad)} parameters):")
for i, g in enumerate(grad):
    print(f"  ∂f/∂θ_{i} = {g:+.4f}")

print(f"\nGradient magnitude: ||∇f|| = {np.linalg.norm(grad):.4f}")
print(f"Circuit evaluations: {2 * len(params_test)} (2 per parameter)")

---

## Section 8: QNN Training Loop

In [ ]:
def train_qnn(
    X_train: np.ndarray,
    y_train: np.ndarray,
    n_qubits: int,
    n_layers: int,
    learning_rate: float = 0.1,
    epochs: int = 50,
    shots: int = 512,
    feature_map: str = 'angle',
    verbose: bool = True
) -> dict:
    """
    Train QNN using gradient descent.
    
    Parameters:
        X_train: Training features (n_samples, n_features)
        y_train: Training labels in {-1, +1}
        n_qubits, n_layers: QNN architecture
        learning_rate: Gradient descent step size
        epochs: Number of training epochs
        shots: Shots per circuit evaluation
        feature_map: 'angle' or 'zz'
        verbose: Print progress
    
    Returns:
        dict: Trained parameters and history
    """
    n_params = 3 * n_qubits * n_layers
    params = np.random.uniform(0, np.pi, n_params)
    
    history = {'loss': [], 'accuracy': []}
    
    for epoch in range(epochs):
        total_loss = 0
        correct = 0
        
        # Accumulate gradients over mini-batch
        gradients = np.zeros(n_params)
        
        for x, y in zip(X_train, y_train):
            # Forward pass
            pred = qnn_expectation(x, params, n_qubits, n_layers, shots, feature_map)
            
            # Loss (MSE)
            loss = (y - pred) ** 2
            total_loss += loss
            
            # Accuracy
            pred_class = 1 if pred > 0 else -1
            if pred_class == y:
                correct += 1
            
            # Gradient (chain rule: ∂L/∂θ = ∂L/∂f * ∂f/∂θ)
            dL_df = -2 * (y - pred)
            grad = compute_gradient(x, params, n_qubits, n_layers, shots, feature_map)
            gradients += dL_df * grad
        
        # Update parameters
        params -= learning_rate * gradients / len(X_train)
        
        # Record history
        avg_loss = total_loss / len(X_train)
        accuracy = correct / len(X_train)
        history['loss'].append(avg_loss)
        history['accuracy'].append(accuracy)
        
        if verbose and epoch % 5 == 0:
            print(f"Epoch {epoch:3d}: Loss = {avg_loss:.4f}, Accuracy = {accuracy*100:.1f}%")
    
    return {
        'params': params,
        'history': history
    }

def evaluate_qnn(
    X_test: np.ndarray,
    y_test: np.ndarray,
    params: np.ndarray,
    n_qubits: int,
    n_layers: int,
    shots: int = 1024,
    feature_map: str = 'angle'
) -> float:
    """
    Evaluate trained QNN on test set.
    
    Returns:
        float: Test accuracy
    """
    correct = 0
    for x, y in zip(X_test, y_test):
        pred = qnn_expectation(x, params, n_qubits, n_layers, shots, feature_map)
        pred_class = 1 if pred > 0 else -1
        if pred_class == y:
            correct += 1
    
    return correct / len(X_test)

print("Note: Full training can take several minutes due to circuit evaluations.")
print("We'll use a small subset for demonstration.")

---

## Section 9: Train QNN on Toy Data

In [ ]:
# Use small subset for faster training
n_train_samples = 20
n_test_samples = 10

data = datasets['moons']
X_train_small = data['X_train'][:n_train_samples]
y_train_small = data['y_train'][:n_train_samples]
X_test_small = data['X_test'][:n_test_samples]
y_test_small = data['y_test'][:n_test_samples]

print(f"Training QNN on {n_train_samples} samples...")
print(f"Configuration: 2 qubits, 2 layers, angle encoding")
print("="*50)

# Train QNN
result = train_qnn(
    X_train_small, y_train_small,
    n_qubits=2, n_layers=2,
    learning_rate=0.3,
    epochs=30,
    shots=256,
    feature_map='angle',
    verbose=True
)

# Evaluate
train_acc = evaluate_qnn(X_train_small, y_train_small, result['params'], 2, 2, 512, 'angle')
test_acc = evaluate_qnn(X_test_small, y_test_small, result['params'], 2, 2, 512, 'angle')

print(f"\n📊 Final Results:")
print(f"  Train accuracy: {train_acc*100:.1f}%")
print(f"  Test accuracy: {test_acc*100:.1f}%")

---

## Section 10: Training Visualization

In [ ]:
# Plot training history
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Loss curve
ax1 = axes[0]
ax1.plot(result['history']['loss'], 'b-', linewidth=2)
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Loss (MSE)')
ax1.set_title('QNN Training Loss')
ax1.grid(True, alpha=0.3)

# Accuracy curve
ax2 = axes[1]
ax2.plot(np.array(result['history']['accuracy']) * 100, 'g-', linewidth=2)
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Accuracy (%)')
ax2.set_title('QNN Training Accuracy')
ax2.grid(True, alpha=0.3)
ax2.axhline(y=50, color='r', linestyle='--', label='Random guess')
ax2.legend()

plt.tight_layout()
plt.show()

---

## Section 11: Classical Comparison

In [ ]:
# Compare with classical SVM
print("Classical Baseline Comparison")
print("="*50)

# Linear SVM
svm_linear = SVC(kernel='linear')
svm_linear.fit(X_train_small, y_train_small)
linear_train_acc = svm_linear.score(X_train_small, y_train_small)
linear_test_acc = svm_linear.score(X_test_small, y_test_small)

# RBF SVM
svm_rbf = SVC(kernel='rbf')
svm_rbf.fit(X_train_small, y_train_small)
rbf_train_acc = svm_rbf.score(X_train_small, y_train_small)
rbf_test_acc = svm_rbf.score(X_test_small, y_test_small)

print(f"\n{'Model':<20} {'Train Acc':<15} {'Test Acc':<15}")
print("-"*50)
print(f"{'QNN (2q, 2L)':<20} {train_acc*100:<15.1f}% {test_acc*100:<15.1f}%")
print(f"{'SVM (Linear)':<20} {linear_train_acc*100:<15.1f}% {linear_test_acc*100:<15.1f}%")
print(f"{'SVM (RBF)':<20} {rbf_train_acc*100:<15.1f}% {rbf_test_acc*100:<15.1f}%")

print("\n⚠️ Note: With more training data and epochs, QNN may improve.")
print("   Classical methods are strong baselines for structured data.")

---

## Section 12: Trap Demonstrations

In [ ]:
print("🚨 TRAP 1: Barren Plateaus")
print("="*50)

# Compute gradient variance for different circuit sizes
def estimate_gradient_variance(n_qubits, n_layers, n_samples=10):
    """Estimate variance of gradients for random parameters."""
    n_params = 3 * n_qubits * n_layers
    x_sample = np.random.uniform(0, np.pi, n_qubits)
    
    gradients = []
    for _ in range(n_samples):
        params = np.random.uniform(0, 2*np.pi, n_params)
        grad = compute_gradient(x_sample, params, n_qubits, n_layers, shots=256)
        gradients.append(grad[0])  # First parameter's gradient
    
    return np.var(gradients)

print("\nGradient variance vs circuit size:")
print(f"{'n_qubits':<12} {'n_layers':<12} {'Var(∂f/∂θ)':<15}")
print("-"*40)

# Small circuit
var_small = estimate_gradient_variance(2, 1, n_samples=5)
print(f"{'2':<12} {'1':<12} {var_small:<15.6f}")

# Medium circuit
var_medium = estimate_gradient_variance(2, 3, n_samples=5)
print(f"{'2':<12} {'3':<12} {var_medium:<15.6f}")

print("\n⚠️ Lesson: Deeper circuits → smaller gradient variance → harder training")
print("   For many qubits, gradients become exponentially small!")

---

## Section 13: Exercises

### 🟢 Exercise 1: Try ZZ Feature Map

Modify the training to use the ZZ feature map instead of angle encoding.

In [ ]:
# TODO: Train QNN with ZZ feature map
# result_zz = train_qnn(
#     X_train_small, y_train_small,
#     n_qubits=2, n_layers=2,
#     feature_map='zz',  # Change this!
#     epochs=30
# )

# YOUR CODE HERE

### 🟡 Exercise 2: Learning Rate Sweep

Try different learning rates and plot the final accuracy vs learning rate.

In [ ]:
# TODO: Sweep learning rates [0.01, 0.1, 0.3, 0.5, 1.0]
# Plot accuracy vs learning rate

# YOUR CODE HERE

### 🔴 Exercise 3: Decision Boundary Visualization

Create a grid of points and classify each to visualize the QNN's decision boundary.

In [ ]:
# TODO: Create meshgrid and classify each point
# xx, yy = np.meshgrid(np.linspace(0, np.pi, 20), np.linspace(0, np.pi, 20))
# predictions = [[qnn_expectation([x, y], result['params'], ...) for x, y in zip(row_x, row_y)] 
#                for row_x, row_y in zip(xx, yy)]
# plt.contourf(xx, yy, predictions)

# YOUR CODE HERE

---

## Section 14: Quick Checks ✅

In [ ]:
print("🧠 Quick Check Questions")
print("="*50)

questions = [
    {
        'q': "1. How many circuit evaluations are needed to compute gradients for 12 parameters?",
        'a': "24 evaluations (2 per parameter for parameter shift rule)"
    },
    {
        'q': "2. What happens to gradient magnitude as you add more qubits?",
        'a': "Gradients become exponentially small (barren plateaus) - variance scales as ~1/2^n for random circuits"
    },
    {
        'q': "3. Why is measurement the 'non-linearity' in QNNs?",
        'a': "Unitaries are linear, but measurement probabilities involve |amplitude|² - a non-linear operation. This lets QNNs approximate non-linear functions."
    },
    {
        'q': "4. When might a QNN outperform classical ML?",
        'a': "When: (1) data is quantum in origin, (2) kernel methods help but classical kernels fail, (3) small data with complex patterns, (4) as part of larger quantum algorithm"
    }
]

for q_dict in questions:
    print(f"\n{q_dict['q']}")
    input("Press Enter to see answer...")
    print(f"Answer: {q_dict['a']}")

---

## Section 15: Summary and Next Steps

### Key Takeaways

1. **QNN = Feature Map + Ansatz + Measurement**
   - Encode data, apply trainable rotations, measure expectation

2. **Parameter Shift Rule gives exact gradients**
   - Evaluate at θ ± π/2, take difference
   - Cost: 2 circuit evaluations per parameter

3. **Barren plateaus limit scalability**
   - Deep + many qubits = vanishing gradients
   - Stay shallow, use problem-specific ansätze

4. **Always compare to classical baselines**
   - SVM, random forest often competitive
   - Quantum advantage not guaranteed

### What You've Implemented

- ✅ Feature maps (angle, ZZ)
- ✅ Variational ansatz with rotations + entanglement
- ✅ Gradient computation via parameter shift
- ✅ Complete training loop with SGD
- ✅ Comparison to classical methods

### What's Next?

- **Advanced**: Quantum kernel methods, QSVM
- **Research**: Provable quantum advantages
- **Applications**: Quantum chemistry, optimization

---

🎉 **Congratulations!** You've completed the Quantum Algorithms module series!